## Probabilistic PCA
Reference: chapter 12, Bishop book

In [106]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from sklearn.model_selection import train_test_split
import os, pickle
import numpy as np
from numpy import log, sum, exp, prod
from numpy.linalg import det
from numpy.random import beta, binomial, dirichlet, uniform, gamma, seed, multinomial, gumbel, rand, multivariate_normal, normal
from scipy.stats import wishart #, norm, randint, bernoulli, beta, multinomial, gamma, dirichlet, uniform
from scipy.special import digamma
from imp import reload
from copy import deepcopy
#import seaborn as sns
import pandas as pd


In [116]:
N = 50000
D = 50
K = 10    # latent dimension M in book
MCsim = 50
burnin = 10

### Generate data: 

In [117]:
# DGP:

sigma2_true = 1.5
z = normal(loc=0, scale=1, size = (K,1)); z.shape
mu_true = uniform(size = (D,1)); mu.shape
W_true = uniform(0,5, size = (D,K)); W_true.shape
mean_true = (W_true @ z + mu_true).squeeze()

X = multivariate_normal(mean = mean_true, cov = sigma2_true * np.eye(D), size = N); X.shape
#X = normal(loc=0, scale=1, size = (N,D)); X.shape

(50000, 50)

In [128]:
#-------------
# Initialize:
#-------------

W = np.empty((D,K,MCsim)) 
sigma2 = np.empty((1,MCsim))           

i = 0
W[:,:,i] = normal(loc=0, scale=1, size = (D,K))
#W[:,:,i] = uniform(0,5, size = (D,K))
sigma2[0,i] = gamma(1)

for i in range(1,MCsim):

    if i % 10 == 0: print("Iter: ",i)

    #--------
    # E-Step
    #--------
    M = (W[:,:,i-1].T) @ W[:,:,i-1] + (sigma2[0,i-1] * np.eye(K))
    M1 = np.linalg.inv(M) 

    x_mean = np.mean(X, axis=0)       # estimator of mu
    X_mean = np.tile(x_mean, (N,1))

    Ez = M1 @ (W[:,:,i-1].T) @ (X - X_mean).T       # 12.54
    
    z_moment2 = np.empty((N,K,K)) ; x_cent = (X - X_mean).T ; cross_pr = np.empty((N,D,K))
    for n in range(N) : 
        
        z_moment2[n,:,:] = sigma2[0,i-1] * M1 + np.dot(Ez[:,n].reshape(K,1),Ez[:,n].reshape(1,K))     
        #cross_pr[n,:,:] = np.dot(x_cent[:,n].reshape(D,1) , Ez[:,n].reshape(1,K))

    Ezz = z_moment2.sum(axis=0)       # 12.55
    #x_Ez = cross_pr.sum(axis=0)

    #----------
    # M-step:
    #----------
    x_Ez = (X - X_mean).T @ Ez.T       # first term in 12.56

    W[:,:,i] = x_Ez @ np.linalg.inv(Ezz)     # 12.56

    # Do summing in 12.57 term-wise:
    first_term = np.sum((X - X_mean)**2)

    # 2nd term...
    second_term = -2*Ez.T.sum(axis=0) @ W[:,:,i].T @ (X - X_mean).sum(axis=0)

    # third term
    third_term = np.trace(z_moment2[i,:,:] @ W[:,:,i].T @ W[:,:,i])

    sigma2[0,i] = (first_term + second_term + third_term)/(N*D)

print('Completed!')    

# Prepare output:
sigma2 = sigma2.squeeze()
#mu = x_mean.squeeze()

Iter:  10
Iter:  20
Iter:  30
Iter:  40
Iter:  50
Completed!


In [133]:
np.mean(sigma2[burnin:])
1e-5

1e-05

In [130]:
W_true

array([[1.91664918, 0.9303516 , 4.53121362, 0.15953354, 1.53891918,
        1.66614403, 0.82038277, 0.16000165, 4.95920888, 2.81036669],
       [1.70633527, 1.1850132 , 2.0457053 , 2.03649816, 3.24004161,
        0.66554828, 3.01854941, 0.2872995 , 0.27022177, 0.56638315],
       [4.49345508, 1.53599043, 0.41350666, 4.36721055, 2.08183964,
        3.38390421, 0.42132157, 4.53500015, 3.78059624, 3.21969283],
       [4.13642575, 3.95835448, 3.99224977, 4.09607926, 3.44219458,
        4.87598998, 2.11202072, 2.42376967, 4.35524208, 4.41426445],
       [1.14955671, 4.77871888, 1.18328525, 1.56357261, 1.00033648,
        4.92624729, 4.6821687 , 2.33499608, 3.22596934, 1.75405219],
       [0.02964314, 0.34311233, 4.71945335, 0.36696857, 4.27145396,
        0.21013555, 4.76547076, 4.28686166, 4.37518134, 4.00842455],
       [0.85223869, 3.35580592, 3.17009716, 2.60645278, 0.21491977,
        2.18828645, 2.37333658, 4.07036074, 1.3253373 , 1.38915686],
       [4.18502068, 4.90681647, 1.9032088

In [131]:
#W[:,:,MCsim-1]#.shape

In [132]:
W.mean(axis=2)

array([[ 1.01805358e-01,  8.83350941e-02, -4.25730501e-02,
         9.52679491e-02,  3.08329373e-02,  1.07084672e-01,
         2.76228481e-02,  1.67171059e-01,  1.17067689e-01,
         5.06621677e-02],
       [ 1.05584408e-01, -3.91968573e-02,  1.23038061e-01,
         9.36381715e-02,  3.56588903e-03,  4.31127533e-02,
         1.07465469e-01,  1.25464508e-01,  3.58101044e-02,
         1.35668884e-01],
       [ 1.55198845e-01,  1.35753711e-01,  4.92273700e-02,
         5.73122346e-02,  4.85884069e-02,  8.21489732e-02,
         9.48407583e-02,  1.36155395e-01,  8.17099019e-02,
         7.41091910e-02],
       [ 1.17880768e-01,  3.80842730e-02,  6.04762867e-02,
         1.39631215e-01,  1.44821653e-02,  3.64940496e-02,
         1.34175521e-01,  1.20451407e-01,  1.10625529e-01,
        -2.46261842e-03],
       [-1.47669615e-02,  4.53385586e-02,  8.44545639e-02,
         4.47794188e-02,  8.95572400e-02,  1.77433195e-01,
         5.91240901e-02,  1.74744999e-01,  1.03242106e-01,
         8.